In [37]:
import os
import csv
import pandas as pd

# Our site DB

In [21]:
import pyodbc
X = 29
Y = 11
Z = 59
conn = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=technionddscourse.database.windows.net;'
                      'DATABASE=ofek0glick;'
                      'UID=ofek0glick;'
                      'PWD=Qwerty12!')
cursor = conn.cursor()
bigserver = pyodbc.connect('DRIVER={SQL Server};'
                      'SERVER=technionddscourse.database.windows.net;'
                      'DATABASE=dbteam;'
                      'UID=dbteam;'
                      'PWD=Qwerty12!')
bigserver_cursor = bigserver.cursor()

In [23]:
bigserver_cursor.execute("select * from categoriestosites")
data = bigserver_cursor.fetchall()
data

[(3, 'ido0z'),
 (4, 'orrubin'),
 (5, 'yarinbs'),
 (6, 'ofri0hefetz'),
 (7, 'rotm'),
 (8, 'roeed'),
 (9, 'd0yehezkel'),
 (10, 'dor0zehavi'),
 (11, 'rubensasson'),
 (12, 'yam0berent'),
 (13, 'dr0lior'),
 (14, 'noam0sprei'),
 (15, 'dahandaniel'),
 (16, 'lior0motola'),
 (17, 'hanna0najjar'),
 (18, 'dmitryk'),
 (19, 'gili0gilad'),
 (20, 'matan0so'),
 (21, 'hilamalka'),
 (22, 'tmyr'),
 (23, 'rompaz'),
 (24, 'michael0ukh'),
 (25, 'eitan0g'),
 (26, 'saharad'),
 (27, 'lanahaj'),
 (28, 'arieln'),
 (29, 'ofek0glick'),
 (30, 'uriel0co'),
 (31, 'noam0shamir'),
 (32, 'eyalmarantz'),
 (33, 'alon0shiri'),
 (34, 'omrio'),
 (35, 'ilanit0sobol'),
 (36, 'ef'),
 (37, 'urimoser'),
 (38, 'yannay0alon')]

### Methods

In [25]:
def drop_tables():
    try:
        cursor.execute("drop table Locks")
        cursor.execute("drop table Log")
        cursor.execute("drop table ProductsOrdered")
        cursor.execute("drop table ProductsInventory")
        cursor.commit()
        print("done")
    except:
        print("nothing to delete")

In [24]:
def create_tables():
    cursor.execute("""CREATE TABLE ProductsInventory (
        productID integer primary key,
        inventory integer ,
        check (inventory >= 0)); """)
    cursor.execute("""CREATE TABLE ProductsOrdered (
        transactionID varchar(30) ,
        productID integer ,
        amount integer,
        check (amount >= 1),
        foreign key(productID) references ProductsInventory(productID)
        on delete cascade,
        primary key(transactionID, productID)
        ); """)
    cursor.execute("""CREATE TABLE Log (
        rowID integer IDENTITY(1,1) primary key,
        timestamp datetime ,
        relation varchar(30),
        transactionID varchar(30),
        productID integer,
        action varchar(20),
        record varchar(2500),
        CHECK(relation IN ('ProductsInventory', 'ProductsOrdered','Locks')),
        CHECK(action IN ('read', 'update','insert','delete')),
        foreign key(productID) references ProductsInventory(productID)
        on delete cascade); """)
    cursor.execute("""CREATE TABLE Locks (
        transactionID varchar(30),
        productID integer ,
        lockType varchar(10),
        CHECK(lockType IN ('read', 'write')),
        foreign key(productID) references ProductsInventory(productID)
        on delete cascade,
        primary key(transactionID,productID,lockType) 
        ); """)
    cursor.commit()
 

### Helper method
Updates the log about an action of aa transaction

In [26]:
def update_log(transactionID,relation, productID,action, record):
    cursor.execute(f"""insert into Log (timestamp,relation,transactionID,productID,action,record) values
                   (CURRENT_TIMESTAMP,{transactionID},{relation},{productID},{action},{record})""")
    cursor.commit()

### Helper method
Given a transactions request for a lock, either grants it and updates the lock relation or doesn't grant it and stops 

In [27]:
def lock_management(transactionID, productID,action,lock_type):
    if action == 'acquire':
        get_locks = f""" select * from Locks where transactionID <> {transactionID} 
                                                      and productID = {productID}"""
        update_log(transactionID, 'Locks',productID, 'read', get_locks)
        cursor.execute(get_locks)
        locks = cursor.fetchall()
        give_lock = f"""insert into Locks (transactionID, productID, lockType) values
                        ({transactionID},{productID},{lock_type})"""
        if len(locks) == 0:
            update_log(transactionID, 'Locks', productID, 'insert', give_lock)
            cursor.execute(give_lock)
            return True
        else:
            if lock_type == 'write':
                return False
            else:
                locks = list(zip(*locks))[2]
                if 'write' in locks:
                    return False
                else:
                    update_log(transactionID, 'Locks', productID, 'insert', give_lock)
                    cursor.execute(give_lock)
                    return True
    else:
        release_lock = f"""DELETE FROM Locks WHERE transactionID={transactionID}
                                                and productID={productID}
                                                and lockType={lock_type}"""
        update_log(transactionID, 'Locks', productID, 'delete', release_lock)
        cursor.execute(release_lock)

In [32]:
def update_inventory(transactionID):
    get_inv = "select * from ProductsInventory"
#     REQUEST READ LOCK!!!!!!!!!!!!!!!!!!!
    cursor.execute(get_inv)
#     RELEASE READ LOCK!!!!!!!!!!!!!!!!!!!
    data = cursor.fetchall()
    inven = [60 if i == 1 else 53 for i in range(1,Y+1)]
#     REQUEST WRITE LOCK!!!!!!!!!!!!!!!!!!!
    if len(data) == 0:
        for i,amount in enumerate(inven):
            query = f"""insert into ProductsInventory (productID, inventory) values
                        ({i+1},{amount})"""
            cursor.execute(query)
    else:
        for i,amount in enumerate(inven):
            query = f"""update ProductsInventory 
                        set inventory = {amount}
                        where productID = {i+1}"""
            cursor.execute(query)
    cursor.commit()
#     RELEASE WRITE LOCK!!!!!!!!!!!!!!!!!!!

In [33]:
drop_tables()

done


In [34]:
create_tables()

In [35]:
update_inventory("ofek")

inserting item 1 with amount 60
inserting item 2 with amount 53
inserting item 3 with amount 53
inserting item 4 with amount 53
inserting item 5 with amount 53
inserting item 6 with amount 53
inserting item 7 with amount 53
inserting item 8 with amount 53
inserting item 9 with amount 53
inserting item 10 with amount 53
inserting item 11 with amount 53


In [36]:
def manage_transactions(T):
    pass

In [ ]:
def build_transaction(order):
    

In [74]:
# os.chdir('orders')
for order in os.listdir():
    with open(order, encoding='utf-8-sig') as curr_order:
        reader = csv.reader(curr_order)
        build_transaction(csv)
        build_query
        

['29', '8', '1']
['29', '9', '1']
['29', '10', '1']
['29', '11', '13']

['29', '1', '50']
['29', '3', '20']
['29', '10', '42']

